In [11]:
import requests
import pandas as pd
import threading
import time
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [12]:
# Dictionary to store names with their respective IP address, login, and password
credentials = {
    'Santa Joana':  {'ip': '10.250.113.97' , 'login': 'WEB', 'password': 'web'},
    # 'Vila Mariana': {'ip': '10.250.115.39' , 'login': 'WEB', 'password': 'web'},
    # 'Ibirapuera':   {'ip': '10.250.110.208', 'login': 'WEB', 'password': 'web'},
    # 'Procriar BH':  {'ip': '192.168.215.43', 'login': 'WEB', 'password': 'web'},
    # 'Brasilia':     {'ip': '192.168.1.20', 'login': 'WEB', 'password': 'web'}
    'Salvador':     {'ip': '10.2.0.71'     , 'login': 'WEB', 'password': 'web'},
}

In [13]:
def get_with_auth(url, login, password):
    # Define the parameters to send in the GET request
    params = {
        'username': login,
        'password': password
    }

    try:
        # Send GET request with parameters
        response = requests.get(url, params=params, verify=False ) #, cert=("C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/cert.pem", "C:/Users/FilipeFurlanBellotti/Documents/Vitrolife/API/API Prototype/cert/key.pem"))
        
        # Raise an exception if the request failed
        response.raise_for_status()
        
        # Return the response content
        return response  # Use response.json() for JSON data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

In [14]:

# Update fetch_patients_for_unit to perform login and use the token in the header for the patients request
def fetch_patients_for_unit(unit_name, creds, results):
    # First, get the token by logging in
    login_url = f"https://{creds['ip']}:4000/LOGIN"
    # params = {
    #     'username': creds['login'],
    #     'password': creds['password']
    # }
    login_response = get_with_auth(login_url, creds['login'], creds['password'])
    if login_response is None:
        results[unit_name] = None
        return
    token = login_response.json().get('Token')
    if not token:
        results[unit_name] = None
        return

    # Now, get patients using the token in the header
    patients_url = f"https://{creds['ip']}:4000/GET/patients"
    headers = {'API-token': token}
    try:
        response = requests.get(patients_url, headers=headers, verify=False)
        response.raise_for_status()
        patients = response.json().get('Patients', [])
        df = pd.DataFrame(patients)
        df['unit_huntington'] = unit_name
        results[unit_name] = df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching patients for {unit_name}: {e}")
        results[unit_name] = None
    
threads = []
results = {}

for unit_name, creds in credentials.items():
    thread = threading.Thread(target=fetch_patients_for_unit, args=(unit_name, creds, results))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

# Combine all dataframes
dfs = [df for df in results.values() if df is not None]
df_all_patients = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
df_all_patients

Error fetching URL: HTTPSConnectionPool(host='10.2.0.71', port=4000): Max retries exceeded with url: /LOGIN?username=WEB&password=web (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018DCE6184F0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
Error fetching URL: HTTPSConnectionPool(host='10.250.113.97', port=4000): Max retries exceeded with url: /LOGIN?username=WEB&password=web (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018DCE618370>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))


""


In [5]:
patients_per_unit = df_all_patients['unit_huntington'].value_counts(dropna=False)
print(patients_per_unit)

unit_huntington
Vila Mariana    2589
Name: count, dtype: int64


In [6]:
# Clean PatientID: add '.' as thousand separator, keep as text
def format_patient_id(pid):
    # Only format if pid is all digits and length >= 4
    if isinstance(pid, str) and pid.isdigit():
        n = len(pid)
        if n > 3:
            return f"{pid[:-3]}.{pid[-3:]}"
        else:
            return pid
    return pid

df_all_patients['PatientID'] = df_all_patients['PatientID'].astype(str).apply(format_patient_id)

In [7]:
df_all_patients['flag_is_test_entry'] = False
df_all_patients.loc[(df_all_patients['PatientID'].str.len() < 5) | (df_all_patients['PatientID'].str.len() > 7), 'flag_is_test_entry'] = True

df_all_patients['flag_is_test_entry'].value_counts(dropna=False)

flag_is_test_entry
False    2579
True       10
Name: count, dtype: int64

In [8]:
df_all_patients[df_all_patients['flag_is_test_entry']]

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth,unit_huntington,flag_is_test_entry
3,PC10T4L77647_43623.5757282639,send mail to Martin,End,Tuesday,NaN,Vila Mariana,True
276,PC10T4L730256_43904.5308762847,"SILVA, RAFAEL D A.",19/05/1982,NaN,NaN,Vila Mariana,True
677,PC10T4L7_44383.4264566435,50031.757,"WALL, KARINA",04/05/1979,1979.05.04,Vila Mariana,True
778,PC10T4L7_44465.4376578356,50034.634,"GALINDO, LUCIENE",03/12/1975,1975.12.03,Vila Mariana,True
808,PC10T4L7_44511.6111412384,50032.607,"HERNANDES, BERNADETE A. P.",16/12/1978,1978.12.16,Vila Mariana,True
1325,PC1R85KM_44939.5469268056,TESTE CRYOTEC,TESTE,CRYOTEC,NaN,Vila Mariana,True
1770,PC1R85KM_45247.6403042130,PH,Meio de,Cultura,NaN,Vila Mariana,True
1999,NEXTGEN_45449.6124767940,test fast placa2,"paciente 1, 2, 4,14,",26,NaN,Vila Mariana,True
2298,PC1R85KM_45687.4841700347,Installation test_HVila_ES8,2024-01-30,NaN,NaN,Vila Mariana,True
2389,PC1R85KM_45449.5615016782,TESTE FAST PLACA,Paciente,1-16,NaN,Vila Mariana,True


In [9]:
patients_per_unit = df_all_patients.loc[~df_all_patients['flag_is_test_entry'], 'unit_huntington'].value_counts(dropna=False)
print(patients_per_unit)

unit_huntington
Vila Mariana    2579
Name: count, dtype: int64


In [12]:
df_all_patients[df_all_patients['PatientID']=='518.379']

,PatientIDx,PatientID,FirstName,LastName,DateOfBirth,unit_huntington,flag_is_test_entry
1510,PC1R85KM_45079.5173685880,518.379,"ARELLANO, FERNANDA M.V.",30/09/1986,1986.09.30,Vila Mariana,False


In [13]:

def fetch_treatments_for_unit(unit_name, creds, patient_ids, results_treatments):
    # First, get the token by logging in
    login_url = f"https://{creds['ip']}:4000/LOGIN"
    login_response = get_with_auth(login_url, creds['login'], creds['password'])
    if login_response is None:
        results_treatments[unit_name] = None
        return
    token = login_response.json().get('Token')
    if not token:
        results_treatments[unit_name] = None
        return

    treatments = []
    headers = {'API-token': token}
    endpoint = f"https://{creds['ip']}:4000/GET/TREATMENT"
    # Process in batches of 10 due to endpoint limit
    for i in range(0, len(patient_ids), 10):
        batch = patient_ids[i:i+10]
        threads_batch = []
        batch_results = [None] * len(batch)

        def fetch_single(idx, pid):
            try:
                params = {'patientIDx': pid}
                resp = requests.get(endpoint, headers=headers, params=params, verify=False)
                resp.raise_for_status()
                data = resp.json().get('Treatments', [])
                for t in data:
                    t['PatientIDx'] = pid
                batch_results[idx] = data
            except Exception as e:
                print(f"Error fetching treatment for {pid} in {unit_name}: {e}")
                batch_results[idx] = []

        for idx, pid in enumerate(batch):
            t = threading.Thread(target=fetch_single, args=(idx, pid))
            threads_batch.append(t)
            t.start()
        for t in threads_batch:
            t.join()
        # Flatten and add to treatments
        for res in batch_results:
            if res:
                treatments.extend(res)
        # Optional: sleep to avoid hammering the server
        time.sleep(0.1)

    # Save results
    results_treatments[unit_name] = pd.DataFrame(treatments) if treatments else pd.DataFrame()

# Prepare patientIDx per unit
unit_patients = df_all_patients[df_all_patients['PatientID']=='518.379']['PatientIDx'].apply(list).to_dict() #df_all_patients.groupby('unit_huntington')['PatientID'].apply(list).to_dict()

threads_treatments = []
results_treatments = {}

for unit_name, creds in credentials.items():
    patient_ids = unit_patients.get(unit_name, [])
    if not patient_ids:
        results_treatments[unit_name] = pd.DataFrame()
        continue
    t = threading.Thread(target=fetch_treatments_for_unit, args=(unit_name, creds, patient_ids, results_treatments))
    threads_treatments.append(t)
    t.start()

for t in threads_treatments:
    t.join()

# Combine all treatments
df_all_treatments = pd.concat([df for df in results_treatments.values() if not df.empty], ignore_index=True) if results_treatments else pd.DataFrame()

# Merge with patients table on PatientIDx
df_patients_treatments = pd.merge(df_all_patients, df_all_treatments, on='PatientID', how='left')
df_patients_treatments

ValueError: No objects to concatenate